# Restructuring Data E-Commerce Customer Support Conversations

conversation column is back-and-forth between agent and customer

source: https://huggingface.co/datasets/NebulaByte/E-Commerce_Customer_Support_Conversations



In [5]:
import pandas as pd

In [6]:
df = pd.read_json('customer_agent.json', lines=True)
df.head()

,issue_area,issue_category,issue_sub_category,issue_category_sub_category,customer_sentiment,product_category,product_sub_category,issue_complexity,agent_experience_level,agent_experience_level_desc,conversation
0,Login and Account,Mobile Number and Email Verification,Verification requirement for mobile number or ...,Mobile Number and Email Verification -> Verifi...,neutral,Appliances,Oven Toaster Grills (OTG),medium,junior,"handles customer inquiries independently, poss...",Agent: Thank you for calling BrownBox Customer...
1,Cancellations and returns,Pickup and Shipping,Reasons for being asked to ship the item,Pickup and Shipping -> Reasons for being asked...,neutral,Electronics,Computer Monitor,less,junior,"handles customer inquiries independently, poss...",Agent: Thank you for calling BrownBox customer...
2,Cancellations and returns,Replacement and Return Process,Inability to click the 'Cancel' button,Replacement and Return Process -> Inability to...,neutral,Appliances,Juicer/Mixer/Grinder,medium,experienced,"confidently handles complex customer issues, e...",Agent: Thank you for calling BrownBox Customer...
3,Login and Account,Login Issues and Error Messages,Error message regarding exceeded attempts to e...,Login Issues and Error Messages -> Error messa...,neutral,Appliances,Water Purifier,less,inexperienced,"may struggle with ambiguous queries, rely on c...","Customer: Hi, I am facing an issue while loggi..."
4,Order,Order Delivery Issues,Delivery not attempted again,Order Delivery Issues -> Delivery not attempte...,negative,Electronics,Bp Monitor,medium,experienced,"confidently handles complex customer issues, e...",Agent: Thank you for contacting BrownBox custo...


### Reformat Conversations

In [65]:
import json

def reformat_conversation(conversation):
    lines = conversation.split('\n\n')
    reformatted = []
    assistant_count = 0
    for line in lines:
        if line.startswith('Agent:'):
            reformatted.append({"role": "assistant", "content": line.replace('Agent: ', '')})
            assistant_count += 1
        elif line.startswith('Customer:'):
            reformatted.append({"role": "user", "content": line.replace('Customer: ', '')})
    
    if assistant_count > 0:
        return reformatted
    else:
        return None

def process_file_line_by_line(input_file_path, output_file_path):
    reformatted_data = []
    
    # process each line as separate json object
    with open(input_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                item = json.loads(line)
                conversation = item.get('conversation', '')
                reformatted_conversation = reformat_conversation(conversation)
                if reformatted_conversation is not None:
                    system_message = {"role": "system", "content": "A polite chatbot that deals with e-commerce products."}
                    reformatted_conversation.insert(0, system_message)
                    reformatted_data.append({"messages": reformatted_conversation})
                else:
                    print("No assistant :(")
            except json.JSONDecodeError:
                print("Error decoding JSON from line:", line)
                continue
    
    # write reformatted conversations to new json file
    with open(output_file_path, 'w', encoding='utf-8') as outfile:
        json.dump(reformatted_data, outfile, ensure_ascii=False, indent=4)

# specify path to input file and desired output file
input_file_path = 'customer_agent.json'
output_file_path = 'converted_conversations.json'

# process file
process_file_line_by_line(input_file_path, output_file_path)

output_file_path

f = open(output_file_path)
data = json.load(f)
with open(output_file_path, 'r') as f:
    with open('converted_conversations.jsonl', 'w') as outfile:
        for entry in data:
            json.dump(entry, outfile)
            outfile.write('\n')
            
        


No assistant :(
No assistant :(


In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()


file_name = client.files.create(
  file=open("converted_conversations.jsonl", "rb"),
  purpose="fine-tune"
)
# print(getattr(file_name, 'id'))

create_job = client.fine_tuning.jobs.create(
  training_file=getattr(file_name, 'id'),
  model="gpt-3.5-turbo"
)

BadRequestError: Error code: 400 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details.', 'type': 'invalid_request_error', 'param': None, 'code': 'exceeded_quota'}}

In [68]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages= [{"role": "system", "content": "A polite chatbot that deals with e-commerce products."}, 
             {"role": "assistant", "content": "Thank you for calling BrownBox Customer Support. My name is Tom. How may I assist you today?"}, 
             {"role": "user", "content": "Hi Tom, I'm trying to log in to my account to purchase an Oven Toaster Grill (OTG), but I'm unable to proceed as it's asking for mobile number or email verification. Can you help me with that?"}, 
             {"role": "assistant", "content": "Sure, I can assist you with that. May I know your registered mobile number or email address, please?"}, 
             {"role": "user", "content": "My registered mobile number is +1 123-456-7890."}, 
             {"role": "assistant", "content": "Thank you. Let me check that for you. I'm sorry to inform you that we don't have this number on our records. Can you please confirm if this is the correct number?"}, 
             {"role": "user", "content": "Oh, I'm sorry. I might have registered with a different number. Can you please check with my email address instead? It's johndoe@email.com."}, 
             {"role": "assistant", "content": "Sure, let me check that for you. (After a few moments) I see that we have your email address on our records. We'll be sending you a verification code shortly. Please check your email and let me know once you receive it."}, 
             {"role": "user", "content": "Okay, I received the code. What do I do with it?"}, 
             {"role": "assistant", "content": "Please enter the verification code in the field provided and click on 'Verify'. Once your email address is verified, you'll be able to proceed with your purchase."}, 
             {"role": "user", "content": "Okay, I entered the code, and it's verified now. Thank you for your help."}, 
             {"role": "assistant", "content": "You're welcome. Is there anything else I can assist you with?"}, 
             {"role": "user", "content": "No, that's all. Thank you."}]

)

print(completion.choices[0].message)

ChatCompletionMessage(content="You're welcome. Have a great day!", role='assistant', function_call=None, tool_calls=None)


In [3]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
client = OpenAI()
# Retrieve the state of a fine-tune
#retrieve_job = client.fine_tuning.jobs.retrieve(getattr(create_job, 'id'))
retrieve_job = client.fine_tuning.jobs.retrieve('ftjob-t7oOstug37ZUbcb6eKy7RSN9')
print(retrieve_job)

FineTuningJob(id='ftjob-t7oOstug37ZUbcb6eKy7RSN9', created_at=1712421372, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9B3P7dz0', finished_at=1712421720, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=8), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-Ke8DBqObuM0u7Zp3BVh771Ph', result_files=['file-1zyBLDrIsP6yabFJ1xTUHY5Y'], status='succeeded', trained_tokens=48550, training_file='file-eiYGcD0x7WZ7v28CxVL0xxGL', validation_file=None, user_provided_suffix=None, seed=1378242904, integrations=[])


In [5]:
completion = client.chat.completions.create(
  model=getattr(retrieve_job, "fine_tuned_model"),
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Thank you for calling BrownBox customer support. My name is Rachel. How may I assist you today?', role='assistant', function_call=None, tool_calls=None)


### Example Chat Completion:
System Description: You are a helpful assistant.

User: Hello!

Generated Assistant Response: Thank you for calling BrownBox customer support. My name is Rachel. How may I assist you today?

In [6]:
completion = client.chat.completions.create(
  model=getattr(retrieve_job, "fine_tuned_model"),
  messages=[
    {"role": "system", "content": "You are a helpful assistant. Your name is [your name here]."},
    {"role": "user", "content": "I have a problem with my toaster. It keeps making weird noses. I want a refund"}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="I'm sorry to hear that. I'll be happy to help you with that. Can you please provide me with your order number?", role='assistant', function_call=None, tool_calls=None)


### Example Chat Completion:
System Description: You are a helpful assistant. Your name is [your name here].

User: I have a problem with my toaster. It keeps making weird noses. I want a refund

Generated Assistant Response: I'm sorry to hear that. I'll be happy to help you with that. Can you please provide me with your order number?

In [7]:
completion = client.chat.completions.create(
  model=getattr(retrieve_job, "fine_tuned_model"),
  messages=[
    {"role": "system", "content": "You are a helpful assistant that writes email responses. Your name is [your name here]."},
    {"role": "user", "content": "I have been waiting for my order for a long time. When will it get here?"}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Hi there, I'm sorry to hear that you have been waiting for your order for a long time. Please provide me with your order number so that I can check the status of your order.", role='assistant', function_call=None, tool_calls=None)


### Example Chat Completion:
System Description: You are a helpful assistant that writes email responses. Your name is [your name here].

User: I have been waiting for my order for a long time. When will it get here?

Generated Assistant Response: Hi there, I'm sorry to hear that you have been waiting for your order for a long time. Please provide me with your order number so that I can check the status of your order.